In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql

In [2]:
conn = sql.connect("chinook.db")

In [3]:
def read(q):
    return pd.read_sql_query(q,conn)

In [4]:
q = """Select * from sqlite_master 
    Where type = 'table'"""

read(q)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


In [11]:
q = """Select * from sqlite_master 
    Where name = 'album'"""

read(q)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...


In [12]:
# employee_name, employee_title, supervisor_name supervisor_tile

q =  """Select e1.first_name || " "|| e1.last_name employee_name, 
            e1.title employee_title, 
             e2.first_name || " "|| e2.last_name supervisor_name,
             e2.title supervisor_title
             from employee e1 
             left join employee e2
             on e1.reports_to =e2.employee_id;"""


read(q)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
2,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager
5,Michael Mitchell,IT Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Laura Callahan,IT Staff,Michael Mitchell,IT Manager


In [14]:
q = """Select first_name || ' '|| last_name customer_name from customer
       Where first_name like '%rob%'"""

read(q)

,customer_name
0,Roberto Almeida
1,Robert Brown


In [13]:
# "Jennefer, jeni"

q = """Select first_name || ' '|| last_name customer_name from customer
       Where first_name like '%jen%'"""

read(q)

,customer_name
0,Jennifer Peterson


In [7]:
# "Jennefer, jeni"

q = """Select first_name || ' '|| last_name customer_name from customer
       Where first_name like 'jen%'"""

read(q)

,customer_name
0,Jennifer Peterson


In [8]:
q = """Select first_name || ' '|| last_name customer_name from customer
       Where first_name like '%jen'"""

read(q)

,customer_name


In [29]:
q = """Select 
         c.first_name || ' '|| c.last_name customer_name,
        count(inv.invoice_id) total_purcahse,
        sum(inv.total) amount_spend
        from customer c
        left join invoice inv
        on c.customer_id = inv.customer_id
        group by 1
        order by 3 Desc
        limit 5
        """
read(q)

,customer_name,total_purcahse,amount_spend
0,František Wichterlová,18,144.54
1,Helena Holý,12,128.70
2,Hugh O'Reilly,13,114.84
3,Manoj Pareek,13,111.87
4,Luís Gonçalves,13,108.90


In [10]:
q = """With
      playlist_info as (Select pl.playlist_id playlist_id,
                             pl.name playlist_name, t.track_id, 
                             (t.milliseconds/1000) length_seconds
                             from playlist pl
                             left join playlist_track plt
                             on plt.playlist_id = pl.playlist_id
                             left join track t
                             on plt.track_id = t.track_id)
                             
        
                             
        
                             
    select * from playlist_info
                             """
read(q)

,playlist_id,playlist_name,track_id,length_seconds
0,1,Music,1.0,343.0
1,1,Music,2.0,342.0
2,1,Music,3.0,230.0
3,1,Music,4.0,252.0
4,1,Music,5.0,375.0
...,...,...,...,...
8714,17,Heavy Metal Classic,2094.0,312.0
8715,17,Heavy Metal Classic,2095.0,295.0
8716,17,Heavy Metal Classic,2096.0,290.0
8717,17,Heavy Metal Classic,3290.0,332.0


In [11]:
q = """With playlist_info as (Select pl.playlist_id playlist_id,
                             pl.name playlist_name, t.track_id, 
                             (t.milliseconds/1000) length_seconds
                             from playlist pl
                             left join playlist_track plt
                             on plt.playlist_id = pl.playlist_id
                             left join track t
                             on plt.track_id = t.track_id)
                             
    select  playlist_id, playlist_name, count(track_id) num_of_tracks,
    sum(length_seconds) length_seconds 
    from playlist_info
    group by 1 """
read(q)

,playlist_id,playlist_name,num_of_tracks,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


In [12]:
q = """Select pl.playlist_id playlist_id,
                             pl.name playlist_name, count(t.track_id) track_id, 
                             sum(t.milliseconds/1000) length_seconds
                             from playlist pl
                             left join playlist_track plt
                             on plt.playlist_id = pl.playlist_id
                             left join track t
                             on plt.track_id = t.track_id
                             group by 1
                             """
read(q)

,playlist_id,playlist_name,track_id,length_seconds
0,1,Music,3290,876049.0
1,2,Movies,0,NaN
2,3,TV Shows,213,500987.0
3,4,Audiobooks,0,NaN
4,5,90’s Music,1477,397970.0
5,6,Audiobooks,0,NaN
6,7,Movies,0,NaN
7,8,Music,3290,876049.0
8,9,Music Videos,1,294.0
9,10,TV Shows,213,500987.0


In [13]:
# """select * from playlist_info
#                              """
# read(q)

In [14]:
q = """ Select c.* from customer c
      left join invoice i
      on i.customer_id = c.customer_id
      group by 1
      having sum(i.total) > 90"""

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
6,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
7,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
8,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
9,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


In [16]:
# q = """Drop view customer_gr_90;"""

# conn.execute(q)

In [17]:
q = """Create view customer_gr_90 as 

      Select c.* from customer c
      left join invoice i
      on i.customer_id = c.customer_id
      group by 1
      having sum(i.total) > 90;"""

conn.execute(q)

In [18]:
q = """Select * from customer_gr_90"""

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
6,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
7,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
8,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
9,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


In [19]:
q = """Create view USA_customers as

      Select * from customer 
     where country = 'USA'
 """

conn.execute(q)

In [20]:
q = """Select *  from USA_customers"""

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
1,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
2,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
3,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
4,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
5,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
6,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
7,23,John,Gordon,None,69 Salem Street,Boston,MA,USA,2113,+1 (617) 522-1333,None,johngordon22@yahoo.com,4
8,24,Frank,Ralston,None,162 E Superior Street,Chicago,IL,USA,60611,+1 (312) 332-3232,None,fralston@gmail.com,3
9,25,Victor,Stevens,None,319 N. Frances Street,Madison,WI,USA,53703,+1 (608) 257-0597,None,vstevens@yahoo.com,5


In [21]:
q = """Select * from sqlite_master 
    Where type = 'table' or type = 'view' """

read(q)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


In [22]:
# UNION, Intersect, Except

In [23]:
q = """Select * from customer_gr_90
       UNION
       select * from USA_customers"""


# {1,2,3} U {3,4,5} # or

# {1,2,3,4,5}

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,16,Frank,Harris,Google Inc.,1600 Amphitheatre Parkway,Mountain View,CA,USA,94043-1351,+1 (650) 253-0000,+1 (650) 253-0000,fharris@google.com,4
6,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
7,18,Michelle,Brooks,None,627 Broadway,New York,NY,USA,10012-2612,+1 (212) 221-3546,+1 (212) 221-4679,michelleb@aol.com,3
8,19,Tim,Goyer,Apple Inc.,1 Infinite Loop,Cupertino,CA,USA,95014,+1 (408) 996-1010,+1 (408) 996-1011,tgoyer@apple.com,3
9,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4


In [24]:
q = """Select * from customer_gr_90
       Intersect
       select * from USA_customers"""


# {1,2,3} n {3,4,5} # and

# {3} 

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
1,20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
2,21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
3,22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4


In [25]:
q = """Select * from customer_gr_90
       Except
       select * from USA_customers"""


# {1,2,3} E {3,4,5} # not

# {1,2} 

read(q)

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
2,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
3,6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
4,13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
5,30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3
6,34,João,Fernandes,None,Rua da Assunção 53,Lisbon,None,Portugal,None,+351 (213) 466-111,None,jfernandes@yahoo.pt,4
7,37,Fynn,Zimmermann,None,Berger Straße 10,Frankfurt,None,Germany,60316,+49 069 40598889,None,fzimmermann@yahoo.de,3
8,42,Wyatt,Girard,None,"9, Place Louis Barthou",Bordeaux,None,France,33000,+33 05 56 96 96 96,None,wyatt.girard@yahoo.fr,3
9,46,Hugh,O'Reilly,None,3 Chatham Street,Dublin,Dublin,Ireland,None,+353 01 6792424,None,hughoreilly@apple.ie,3


In [26]:
q = """With all_customers as (
       Select c.first_name || " " || c.last_name customer_names,
       c.country,
       sum(i.total) total_spend
       from customer c
       left join invoice i
       on i.customer_id = c.customer_id
       group by 1
         )
         
select * from all_customers
         """
read(q)

,customer_names,country,total_spend
0,Aaron Mitchell,Canada,70.29
1,Alexandre Rocha,Brazil,69.30
2,Astrid Gruber,Austria,69.30
3,Bjørn Hansen,Norway,72.27
4,Camille Bernard,France,79.20
5,Daan Peeters,Belgium,60.39
6,Dan Miller,USA,95.04
7,Diego Gutiérrez,Argentina,39.60
8,Dominique Lefebvre,France,72.27
9,Eduardo Martins,Brazil,60.39


In [27]:
q = """With all_customers as (
       Select c.first_name || " " || c.last_name customer_names,
       c.country country,
       sum(i.total) total_spend
       from customer c
       left join invoice i
       on i.customer_id = c.customer_id
       group by 1
         )
         
select country, customer_names,  Max(total_spend) top_spend
from all_customers
group by 1

order by 3 Desc
limit 5
         """
read(q)

,country,customer_names,top_spend
0,Czech Republic,František Wichterlová,144.54
1,Ireland,Hugh O'Reilly,114.84
2,India,Manoj Pareek,111.87
3,Brazil,Luís Gonçalves,108.90
4,Portugal,João Fernandes,102.96
